# Фильтры калмана на примерах

## Одномерный случай

### Постановка задачи

Допустим у нас есть человек, стоящий на месте и желающий определить свое место положения. В текущий момент времени ему доступны два источника GPS и ГЛАНАС. Эти источники не точны и присылают координату с некоторой погрешностью n1 для GPS и n2 для ГЛАНАС соответственно. Из технической документации нам известно, что источник GPS в среднем имеет ошику $\sigma_1$, а источник ГЛАНАС $\sigma_2$. Причём эти ошибки имеют нормальное распределение с нулевым математическим ожиданием, то есть они в среднем дают правильный ответ. Задача: используя эти два наблюдения найти более точную оценку координаты X.

Итак, опишем задачу более формально. Координату, полученную от GPS обозначим как $z_1$, она представляет собой истинное значение координаты $x$ плюс некоторая случайная величина (шум) $n_1$ с диспепсией $\sigma_1^2$ и нулевым математическим ожиданием

$z_1=x+n_1$ $n_1\sim N(0,\sigma_1^2)$

Аналогично для второго есть источника

$z_2=x+n_2$ $n_2\sim N(0,\sigma_2^2)$

![two_sourcre](images/kalman_filter_algorithm/image001.png)

Задача: построить алгоритм объединяющий эти два наблюдения и дающий более точную оценку координаты X

### Основная идея

Давай  порассуждаем. Мы не знаем точного значения координаты Х, он точно знаем что она, с одной стороны, лежит в диапазоне $[z_1-\sigma_1, z_1+\sigma_1]$, с другой стороны ее диапазон равен $[z_2-\sigma_2,z_2+\sigma_2]$. Причём, наиболее вероятным значением X, с точки зрения первого источника является $z_1$, а с точки зрения второго источника $z_2$. Хотелось бы построить алгоритм, принимающий два этих диапазона и учитывающий, что наиболее вероятными значениями является или $z_1$, или $z_2$. Как построить такой алгоритм ?

## Обзор фильтра Калмана

Давайте посмотрим на фильтр Калмана как на черный ящик. Фильтр Калмана имеет входы и выходы. Входные данные зашумлены и иногда дают неточные измерения. Результаты его представляют собой менее зашумленные и иногда более точные оценки. Оценками могут быть параметры состояния системы, которые не измерялись и не наблюдались. Последнее предложение описывает суперсилу фильтра Калмана. Фильтр Калмана оценивает параметры системы, которые не наблюдаются и не измеряются.

Короче говоря, вы можете думать о фильтре Калмана как об алгоритме, который может оценивать наблюдаемые и ненаблюдаемые параметры с большой точностью в режиме реального времени. Оценки с высокой точностью используются для прогнозов и решений. По этим причинам фильтры Калмана используются в робототехнике и системах реального времени, которым необходима надежная информация.

Проще говоря, фильтр Калмана — это универсальный алгоритм, который используется для оценки параметров системы. Он может использовать неточные или зашумленные измерения для более точной оценки переменной или находить значения вообще не наблюдаемой переменной.

Реальную популярность фильтр Калмана обрел не благодаря способности сглаживать измерения, а благодаря своей способности оценивать параметры системы, которые невозможно измерить или наблюдать.

## Описание алгоритма 

хорошо описан <a href="https://habr.com/ru/articles/594249/">тут</a>

### Подключение библиотек

In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import numpy as np
import random

### Постановка задачи

Допустим у нас есть робот с датчиком на голове. Датчик возвращает кординаты Х и У через равные промежутки времени. Показания дата не точны и имеют дисперсию ss. Робот движется с постоянной не известной нам скоростью. На его координаты также влияют внешние факторы, его колеса  могут проскальзывать или застревать в грязи. Смоделируем это как шум с дисперсией ms. Задача найти скоростью с помощью фильтра Калмана. 

### Класс робот

In [2]:
class Robot:
    def __init__(self, ss, ms):
        self.ss = ss
        self.ms = ms
        self.vx = random.randint(1, 10)
        self.vy = random.randint(1, 10)
        self.x = random.randint(0, 10)
        self.y = random.randint(0, 10)

    def sn(self):
        return np.random.normal(0, self.ms, 1)[0]

        
    def move(self):
        self.x = self.vx + self.x + self.sn()
        self.y = self.vy + self.y + self.sn()

    

In [3]:
r = Robot(0.5,0.3); r.x; 

In [4]:
r.x

10

In [5]:
r.vx

6

In [6]:
r.move()

In [7]:
r.x

16.131088884411458